Mustafa Yetişir
10/Jan/2023

Lucas Kanade optical flow algorithm.

In [28]:
import cv2
import numpy as np


def video_to_numpy(path: str) -> np.ndarray:
    """ Convert the video frames into a numpy array

    Args:
        path (str): path of the video

    Returns:
        np.ndarray: 3D numpy array of shape (T, H, W)
    """
    cap = cv2.VideoCapture(path)

    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frames = []

    ret, frame = cap.read()
    while ret:
        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img_arr = grayFrame[::3, ::3]
        frames.append(img_arr)
        ret, frame = cap.read()

    return np.stack(frames).astype(np.uint8)


In [ ]:
from typing import Tuple
import numpy as np

from previous_homework import sobel_horizontal, sobel_vertical, gaussian_filter


WINDOW_SIZE = 3  # Determine a value
THRESHOLD = 0.8 # Determine a value

image_sequence = video_to_numpy("video.mp4")
u_sequence = np.zeros(image_sequence.shape, dtype=np.float32)
v_sequence = np.zeros(image_sequence.shape, dtype=np.float32)


def derivatives(img: np.ndarray, next_img: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """ Calculate derivative images.

    Args:
        img (np.ndarray): 2D gray video frame of shape (H, W)
        next_img (np.ndarray): 2D next gray video frame of shape (H, W)

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray]:
            - x derivative I_x of shape (H, W)
            - y derivative I_y of shape (H, W)
            - temporal derivative I_t of shape (H, W)
    """
    #we are calculating derivatives by using sobel operators we created for previous homeworks
    x_derivative = sobel_horizontal(img)
    y_derivative = sobel_vertical(img)
    #calculating time derivative by making the subtraction below 
    time_derivative = next_img - img
    mytuple = (x_derivative,y_derivative,time_derivative)
    #return the tuple that has derivatives
    return mytuple 


def lucas_kanade(x_derivative: np.ndarray,
                 y_derivative: np.ndarray,
                 time_derivative: np.ndarray,
                 window_size: int,
                 threshold: float
                 ) -> np.ndarray:
    """ Lucas Kanade optical flow for single frame transition

    Args:
        x_derivative (np.ndarray): x derivative I_x of shape (H, W)
        y_derivative (np.ndarray): y derivative I_y of shape (H, W)
        time_derivative (np.ndarray): temporal derivative I_t of shape (H, W)
        window_size (int): Window size of W_p (square windows)
        threshold (float): Eigen value threshold of the covariance matrix A^T A 

    Returns:
        np.ndarray: flow matrix of shape (H, W, 2) containing x and y flows.
    """

    flow = np.zeros((x_derivative.shape[0], x_derivative.shape[1], 2))
    #we are creating step variables so that in the for loops we don't need to think about window going outside 
    step_x_horizontal = x_derivative.shape[0] - window_size + 1
    step_x_vertical = x_derivative.shape[1] - window_size + 1

    for i in range(step_x_horizontal):
        for j in range(step_x_vertical):
            sub_x = x_derivative[i:i+window_size, j:j+window_size].flatten()
            sub_y = y_derivative[i:i+window_size, j:j+window_size].flatten()
            sub_t = time_derivative[i:i+window_size, j:j+window_size].flatten()
            #calculating x, y and time derivatives inside the window by using window_size and derivative inputs 
            A = np.stack([sub_x, sub_y], axis=1)
            eigen_values, _ = np.linalg.eig(A.T @ A) 
            # we are calculating the eigenvalues in order to check later if we will think about related point for flow or not

            if np.max(eigen_values) < threshold:#if eigenvalues are not proper for checking flow, continue
                continue

            flow[i, j] = np.linalg.pinv(A.T @ A) @ A.T @ sub_t
            # u = (A^T * A)^-1 * A^T * B     where A contains spatial derivatives and B contains time derivatives 

    return flow


for index in range(len(image_sequence) - 1):

    x_derivative, y_derivative, time_derivative = derivatives(
        image_sequence[index], image_sequence[index + 1])

    uv_values = lucas_kanade(
        x_derivative, y_derivative, time_derivative,
        window_size=WINDOW_SIZE, threshold=THRESHOLD)

    u_sequence[index] = uv_values[:, :, 0]
    v_sequence[index] = uv_values[:, :, 1]


### Run the cell below to visualize your implementation

In [ ]:
from visualizers.flow import FlowRenderer

FlowRenderer(image_sequence,
             u_sequence,
             v_sequence)()


- Why can we not reliably compute flow for windows $\mathcal{W}_p$ with small eigen values?

The reason we are looking for high eigen values is that it is important to detect corners and choose the windows on the corners mostly. The reason behind this choosing corners is explained in the aperture problem question below, but it can be simply explained like if we want to compute the flow without being stick to only one direction we choose corners to monitor. If we choose windows with both eigen values are high, it is much more likely to choose a corner, if only one is high and other one is still small we choosed an edge and it is still not were smart to do. We want to detect edges and compute flow on those windows, and high eigen values is important for this operation. 

- Explain the aperture problem.

The aperture problem is; if we are monitoring an image by using an aperture, the motion direction of an object in the image or some of the features in the image may be indefinite and ambiguous. 

If we think that we are trying to observe a rectangle and it is moving in a horizontal line, if we will look at the area of it's horizontal edge we will not be able to understand the motion. If the rectangle is moving both in a horizontal and vertical lines, we won't be seeing one of those motion ways depending on the edge we are looking at by our aperture, and think that the object is moving only on the one way. This is called the aperture problem, and in order to solve it we need to check corner points, so that we can see the move in any side. That is why we are finding features and look at those features in Lucas Kanade, we can't look at the whole image at once, but we also need to arrange our aperture wisely so we look at the chosen features.

The one perfect example for this problem is the barber pole illusion.

- Are image corners well-conditioned points for optical flow? Show your answer.

As I explained in the question above, corner points are good for operations like optical flow estimation or motion estimation, because in corner points are the regions with (at least) 2 different directions of the gradient. 